## to do some day - 
- update information about this notebook and what it does 

Merger definition for this project: 
 - RG 2015, 2017, Patton 2020, Ventou 2020 all use same definition of merging:
     - when a subhalo has two or more direct progenitors, the progenitors "merged"
     - can check the subhalo descendent ID's and find the first snapshot where they are the same? (would this cover places where the subhalo descendent has a progenitor that skips a snapshot? -> can check by doing Root Descendant ID check as well.)
     

to do: 
- quantify how many pairs will merge in the future
- quantify the fraction that merge as a function of the separation of the pair at z=1, 1.5, 2
- need to pull out all pairs at z=1, 1.5, 2 and their orbits


"merged" snapshot after the descendant ID is the same number at a single snapshot

## To do for Oct. 10:
- make function to calculate the fraction for a given redshift
- find the merger fractions for z=1,1.5,2
- save the data for the merger fractions

- think about next steps, i.e. saving separations
- think about how to quantify the fraction that merge as a function of the separation (bin in separation at the redshift, find fraction of things in that separation that merge)

**Done:**
- implement merger definition for one pair in z=1.5
- check for easiest way to define (and when?) 
- get merged fraction for z=1.5 via RootDescendantID method
- get merged fraction for z=1.5 via merger definition

In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import sys
import importlib

from utils_orbs.orb_paths import SetupPaths
import utils_orbs.readsubfHDF5Py3 as readSub
# from utils.read_group_cats import ReadCats
from utils_orbs.merger_trees import TraceMergerTree
from utils_orbs.readMergerHistoryFiles import FindHistory
from utils_orbs.vectorCorrection import vectorCorrection as vector

paths = SetupPaths()

imported h5py...
modules seem to have loaded?


In [3]:
plt.show();
plt.rcParams.update({'font.size':20,"xtick.direction":"in","ytick.direction":"in", 
                     "xtick.top":True, "ytick.right":True,"text.usetex":False,
                     "xtick.labelsize":18,"ytick.labelsize":18})

Goal: 
- get first z=1.5 pair (snapshot 40), and pull out matched lists of descendant info

In [9]:
# assign all the data to a dictionary and close hdf5 file
f = h5py.File(f"../data/lowmass_z1p5.hdf5", 'r')
pairs = {}
for key, val in f.items():
    if key == "Header":
        continue
    pairs[key]=np.array(val)
f.close()
    

In [8]:
# grab subfind id's of first pair
pair1 = pairs['Sub1 ID'][0]
pair2 = pairs['Sub2 ID'][0]

mass1 = pairs['Sub1 Mass'][0]
mass2 = pairs['Sub2 Mass'][0]

In [26]:
# get the merger trees for each subhalo and make sure the mass makes sense
# note: all values returned by the merger trees past, future, and merged 
#     branches are in simulation units
treedict = {}
for sub in [pair1,pair2]:
    treedict[sub] = TraceMergerTree(snapshot=40,physics="hydro",sim="TNG",subfindID=sub)

In [28]:
# converting to physical units for TNG and making sure masses are correct at z=1.5
little_h = 0.6774

print(mass1,treedict[pair1].mergedbranch['SubhaloMass'][treedict[pair1].mergedbranch['SnapNum']==40][0]/little_h)
print(mass2,treedict[pair2].mergedbranch['SubhaloMass'][treedict[pair2].mergedbranch['SnapNum']==40][0]/little_h)

35.549774260023966 35.549774260023966
27.455131951988992 27.455131951988992


- 'DescendantID'
- 'FirstProgenitorID'
- 'FirstSubhaloInFOFGroupID'
- 'LastProgenitorID'
- 'MainLeafProgenitorID'
- 'Mass'
- 'MassHistory'
- 'NextProgenitorID'
- 'NextSubhaloInFOFGroupID'
- 'NumParticles'
- 'RootDescendantID'
- 'SnapNum'
- 'SubfindID'

If I want to make a loop that goes through and figures out the location/snapshot of merger, I can FIRST check for RootDescendantID to see if they are the same, and if they are, then they should have the same Descendant ID or Last Progenitor ID up until the snapshot where they merged (i.e. I can count them)

In [39]:
# this is where the "previous progenitor" is the same (meaning that they have merged at snapshot 38)
np.where(treedict[pair1].mergedbranch['LastProgenitorID'][0:50] == treedict[pair2].mergedbranch['LastProgenitorID'][0:50])


(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38]),)

In [44]:
# this is where the "next descendent" is the same (so they will merge at the second to last snapshot)
np.where(treedict[pair1].mergedbranch['DescendantID'][0:50] == treedict[pair2].mergedbranch['DescendantID'][0:50])


(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39]),)

In [61]:
print("The first snapshot AFTER merger is:",treedict[pair1].mergedbranch['SnapNum'][np.where(treedict[pair1].mergedbranch['DescendantID'][0:50] == treedict[pair2].mergedbranch['DescendantID'][0:50])][-1])
print("The first snapshot before merger is:",treedict[pair1].mergedbranch['SnapNum'][np.where(treedict[pair1].mergedbranch['LastProgenitorID'][0:50] == treedict[pair2].mergedbranch['LastProgenitorID'][0:50])][-1])
print("The first snapshot before merger is:",treedict[pair1].mergedbranch['SnapNum'][np.where(treedict[pair1].mergedbranch['SubfindID'][0:50] == treedict[pair2].mergedbranch['SubfindID'][0:50])][-1])



The first snapshot AFTER merger is: 60
The first snapshot before merger is: 61
The first snapshot before merger is: 61


___

## Calculating full merger fraction for z=1.5

**Notes:**

I can use the Descendant ID to identify the snapshot where the two subhalos will next merge. 

Now let's try to go through each of the pairs, and I will save 3 things: 
- Whether the pair merges via the 'RootDescendantID'
- Whether the pair merges via the 'DescendantID'
- What the snapshot after the 'DescendantID' equivalence happens

The below script first checks if the RootDescendantID of the primary and secondary of each pair is the same (meaning they merge at some point in the simulation). If they do merge, mask the snapshots where both subhalos exist simultaneously and determine when they have the same descendant ID, and output the "merger snapshot" defined as the snapshot of the subhalo that results from merger. 

*Might need to check if the groupID is the same at all previous snapshots (meaning they are "infalling") or to figure out how long it has been since infall.*

In [105]:
merger_root, merger_desc, snap_merged = np.zeros((3,len(pairs['Sub1 ID'])))

for ind in range(len(pairs['Sub1 ID'])):
    primary_id = pairs['Sub1 ID'][ind]
    secondary_id = pairs['Sub2 ID'][ind]
    
    treedict = {}
    for sub in [primary_id,secondary_id]:
        treedict[sub] = TraceMergerTree(snapshot=40,physics="hydro",sim="TNG",subfindID=sub)

    tree_primary = treedict[primary_id].mergedbranch
    tree_secondary = treedict[secondary_id].mergedbranch
    
    # check if root descendant ID is the same between the two halos
    root1 = tree_primary['RootDescendantID'][0]
    root2 = tree_secondary['RootDescendantID'][0]
    check_root = root1 == root2
    
    if check_root:
        merger_root[ind] = True
    
        # make masks to check descendants 
        snaps1 = tree_primary['SnapNum']
        snaps2 = tree_secondary['SnapNum']

        mask1 = np.isin(snaps1, snaps2)
        mask2 = np.isin(snaps2, snaps1)

        desc1 = tree_primary['DescendantID'][mask1]
        desc2 = tree_secondary['DescendantID'][mask2]
        
        try:
            ind_premerge = np.where(desc1==desc2)[0][-1]
            ind_postmerge = np.where(desc1==desc2)[0][-2]
            snap_premerge = tree_primary['SnapNum'][mask1][ind_premerge]
            snap_postmerge = tree_primary['SnapNum'][mask1][ind_postmerge]
            
            merger_desc[ind] = True
            snap_merged[ind] = snap_postmerge
        
        except IOError:
            print('Error in the snapshot number')
            
merger_dict = {"MergersRoot":merger_root,"MergersDescendant":merger_desc,"MergerSnapshot":snap_merged}
merger_dict

{'MergersRoot': array([1., 0., 1., ..., 1., 1., 1.]),
 'MergersDescendant': array([1., 0., 1., ..., 1., 1., 1.]),
 'MergerSnapshot': array([61.,  0., 73., ..., 43., 41., 44.])}

In [114]:
mergerfrac_root = np.count_nonzero(merger_dict['MergersRoot']) / len(merger_dict['MergersRoot'])
mergerfrac_desc = np.count_nonzero(merger_dict['MergersDescendant']) / len(merger_dict['MergersDescendant'])
print(f"Merger fraction defined based on the RootDescendantID: {mergerfrac_root:.3f}")
print(f"Merger fraction defined based on the Descendant ID: {mergerfrac_desc:.3f}")
print(f"Average snapshot of mergers: {np.mean(merger_dict['MergerSnapshot'][merger_dict['MergerSnapshot']!=0]):.3f}")


Merger fraction defined based on the RootDescendantID: 0.901
Merger fraction defined based on the Descendant ID: 0.901
Average snapshot of mergers: 50.901


## Creating merger fraction at z=1,1.5,2

## Merger fraction as function of separation at z=1,1.5,2